# **ABDULLAH ALSHADADI - 190582184**

Computer Science (Software engineering) Bsc Student

## Dissertation Title:
----------------------
## Machine Learning (ML) Model for The Centre for Search Research (TCSR) to Find Missing Persons Using Drones Imaging Data.

In [ ]:
!pip install --upgrade mlxtend

In [ ]:
import mlxtend
print(mlxtend.__version__)

In [ ]:
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from numpy import ndarray
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import seaborn as sns
import pandas as pd
import numpy as np
import pathlib
import datetime
import shutil
import cv2
import os

%load_ext tensorboard
%matplotlib inline

# Demonstration Purposes Only
# from google.colab.patches import cv2_imshow

In [ ]:
class SubdivisionIsBiggerThanBatch(Exception):
    def __init__(self, message, errors=None):
        self.message = message
        self.errors = errors
        super(SubdivisionIsBiggerThanBatch, self).__init__(message)

Enable Google Drive for Uploading and Executing the Program

In [ ]:
from google.colab import drive
DRIVE_PATH = "/content/drive/"
drive.mount(DRIVE_PATH)
DRIVE_PATH = DRIVE_PATH + "MyDrive/"

## **Select Model to Run By Typing True**

In [ ]:
#################
# MODELS TO RUN #
#################
YOLO_MODEL = True
RES_NET_MODEL = True
MOBILENET_MODEL = True

In [ ]:
##########
# SET-UP #
##########
SKIP_DATA_PROCESSING = False
CLASSES = ["Person", "No Person"]
PATH_DATA = DRIVE_PATH + "quick_preview/"
TEST_DATA = PATH_DATA + "test/"
FINAL_MODELS_PATH = PATH_DATA + "models/"
BACKUP_TENSOR = PATH_DATA + "backup/tensorflow/"
NUMPY_SAVE = PATH_DATA + "numpy/"
NUM_BUF = 30
MIN_CONTOUR = 100
MAX_CONTOUR = 10000
# Use website to visualise how the HSV range colour selected looks  like:
# https://wamingo.net/rgbbgr/ (WARNING: it uses (360Degree, 100%, 100%) data)
# OpenCV uses (0-179, 0-255, 0-255)
#
# Trick to convert it:
# (half the degree, 255 x 1.0, 255 x 1.0)
# (Trick got from https://stackoverflow.com/a/10951189)
COLOUR_HSV_RANGE = [  # [lower bound, upper bound]
    [np.array([156, 148, 150]), np.array([179, 255, 255])],  # Red Range
    [np.array([110, 125, 125]), np.array([150, 255, 255])]  # Blue Range
]

if os.path.isdir(PATH_DATA) and os.path.exists(PATH_DATA):
  pass
else:
  raise FileNotFoundError("Path to Directory " + PATH_DATA + " Not Found")

SETUP_DATA_DIRS = [TEST_DATA, FINAL_MODELS_PATH, BACKUP_TENSOR, NUMPY_SAVE]
for setup_data_dir in SETUP_DATA_DIRS:
  if os.path.isdir(setup_data_dir) and os.path.exists(setup_data_dir):
    print(setup_data_dir)
    pass
  else:
    pathlib.Path(setup_data_dir).mkdir(parents=True, exist_ok=True)
    print(setup_data_dir)

In [ ]:
####################
# Hyper-parameters #
####################
IMG_SIZE = 416
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
BATCH_SIZE = 20
SHUFFLE_BUFFER_SIZE = 100
EPOCHS = 10
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
###############
# YOLO SET-UP #
###############
# Directories
YOLO_BACKUP = PATH_DATA + "backup/yolo/"
YOLO_FRAMES = PATH_DATA + "frames/"
YOLO_METADATA = PATH_DATA + "metadata/yolo/"

# Files
YOLO_MAIN_DATA = YOLO_METADATA + "main.txt"
YOLO_VALID_DATA = YOLO_METADATA + "valid.txt"
YOLO_TEST_DATA = YOLO_METADATA + "test.txt"
YOLO_DATA_FILE = YOLO_METADATA + "obj.data"
YOLO_NAMES_FILE = YOLO_METADATA + "obj.names"

# Make the directories
YOLO_PATHS = [YOLO_BACKUP, YOLO_FRAMES, YOLO_METADATA]
for yolo_path in YOLO_PATHS:
  if os.path.isdir(yolo_path) and os.path.exists(yolo_path):
    print(yolo_path)
    pass
  else:
    pathlib.Path(yolo_path).mkdir(parents=True, exist_ok=True)
    print(yolo_path)

# *.data file
file_data = open(YOLO_DATA_FILE, "w")
file_data.write("classes={0}\n"
                "train={1}\n"
                "valid={2}\n"
                "names={3}\n"
                "backup={4}\n"
                .format(len(CLASSES),
                        YOLO_MAIN_DATA,
                        YOLO_VALID_DATA,
                        YOLO_NAMES_FILE,
                        YOLO_BACKUP))
file_data.close()
# *.names file
file_names = open(YOLO_NAMES_FILE, "w")
for name in CLASSES:
    file_names.write("{0}\n".format(name))
file_names.close()

In [ ]:
#########################
# YOLO Hyper-parameters #
#########################
YOLO_BATCH = 64
YOLO_SUBDIVISION = 32
YOLO_MAX_BATCHES = 4000
YOLO_LOWER_STEPS = 400
YOLO_UPPER_STEPS = 450
if YOLO_SUBDIVISION > YOLO_BATCH:
    raise SubdivisionIsBiggerThanBatch(str(YOLO_SUBDIVISION) + " SUBDIVISION is bigger than " + str(YOLO_BATCH) + " BATCH")

# DATA PROCESSING

Progress bar to see the processing progress

In [ ]:
# Code extracted from:
#
# https://gist.github.com/greenstick/b23e475d2bfdc3a82e34eaa1f6781ee4
def print_progress_bar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█', autosize=False):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        autosize    - Optional  : automatically resize the length of the progress bar to the terminal window (Bool)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    styling = '%s |%s| %s%% %s' % (prefix, fill, percent, suffix)
    if autosize:
        cols, _ = shutil.get_terminal_size(fallback=(length, 1))
        length = cols - len(styling)
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print('\r%s' % styling.replace(fill, bar), end='\r')
    # Print New Line on Complete
    if iteration == total:
        print()

## **Functions to Run Data processing**

Masking function

In [ ]:
def mask_colour_range(frame: ndarray) -> ndarray:
    """
    Using the HSV colour range to create the masking value for a frame to be masked;
    checkout COLOUR_HSV_RANGE above in the setup section
    @params:
        frame - Required : the current frame that going to be masked (ndarray)
    @returns:
        The mask frame value (ndarray)
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = 0
    for bounding_range in COLOUR_HSV_RANGE:
        #                              lower bound      , upper bound
        mask = mask + cv2.inRange(hsv, bounding_range[0], bounding_range[1])
    return mask

The heart of the data processing: the loading of the data and data processing

In [ ]:
def load_video(path: str, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    """
    Loads the video and store into numpy array
    @params:
        path       - Required                     : location of the video file to load (Str)
        max_frames - Default 0                    : used to break if frames is not present (Int)
        resize     - Default (IMG_SIZE, IMG_SIZE) : uses the hyperparameter to resize the frame (Tuple(Int, Int))
    @returns:
        Tuple of 2 lists, one for training data and the other is training labels (Tuple(List, List))
    """
    cap = cv2.VideoCapture(path)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    frames = []

    frames_with_missing_person = []
    labels_for_missing_person = []

    frames_without_missing_person = []
    labels_for_without_missing_person = []

    progress = 0
    try:
        # while len(frames) <= 6000:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            # Clean copy of frame
            temp_frame = frame.copy()
            frames.append(frame)

            #############################
            # Labels the missing person #
            #############################
            mask = mask_colour_range(frame)

            kernel = np.ones((30, 15), np.float32) / 225
            mask = cv2.filter2D(mask, -1, kernel)

            result = cv2.bitwise_and(frame, frame, mask=mask)

            if len(frames) % NUM_BUF == 0 or total_frames - len(frame) == total_frames % NUM_BUF:
                contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if len(contours) != 0:
                    frames_with_missing_person.append(temp_frame)
                    for label in CLASSES:
                        label_name = "Person"
                        if label == label_name:
                            labels_for_missing_person.append(CLASSES.index(label_name))
                            print("Found:", label_name, ", Label Num:", CLASSES.index(label_name))
                    proces_images(path, contours, frame, temp_frame, progress)
                else:
                    frames_without_missing_person.append(temp_frame)
                    for label in CLASSES:
                        label_name = "No Person"
                        if label == label_name:
                            labels_for_without_missing_person.append(CLASSES.index(label_name))
                            print("Found:", label_name, ", Label Num:", CLASSES.index(label_name))
                # Demonstration Purposes Only
                # cv2.imshow("image", np.hstack([frame, result]))
                # cv2_imshow(np.hstack([frame, result]))
            #############################

            # Demonstration Purposes Only
            # cv2.imshow("image", np.hstack([frame, result]))
            # cv2_imshow(np.hstack([frame, result]))

            if cv2.waitKey(1) == ord('q'):
                break
            if len(frames) == max_frames:
                break

            # Progress bar to see the progress of labelling the missing person
            progress += 1
            if path != 0:
                print_progress_bar(progress, total_frames,
                                   prefix='Progress:', suffix='Extracting Images',
                                   autosize=True)
    finally:
        cap.release()
    training_data = frames_with_missing_person + frames_without_missing_person
    training_labels = labels_for_missing_person + labels_for_without_missing_person
    return training_data, training_labels

In [ ]:
def proces_images(path: str, contours: list, current_frame: ndarray, clean_frame: ndarray, progress: int):
    """
    Adds contours to the masked areas of a frame and if YOLO_MODEL = True, it saves the frames
    as *.jpg files with the coordinates of the contours' rectangle area saved as a *.txt file
    @params
        path          - Required : path to the video (Str)
        contours      - Required : list of contours to be used to find the counterArea() and boundingRect(contours) (List)
        current_frame - Required : the frame that is going to be processed with contours (ndarray)
        clean_frame   - Required : clean version of the current_frame that has not been processed for saving (ndarray)
        progress      - Required : the progress count of the frames loaded in to differentiate file names when saving (Int)
    """
    for contours in contours:
        if MIN_CONTOUR < cv2.contourArea(contours) < MAX_CONTOUR:
            x, y, w, h = cv2.boundingRect(contours)

            if YOLO_MODEL is True:
                file_without_extension = os.path.splitext(os.path.basename(path))[0]
                frame_file_name = "{0}{1}-{2}".format(YOLO_FRAMES, file_without_extension, progress)

                if os.path.isfile(frame_file_name + ".jpg") is False:
                    cv2.imwrite(frame_file_name + ".jpg", clean_frame)

                # Coordinates data
                if os.path.isfile(frame_file_name + ".txt"):
                    coord_file = open(frame_file_name + ".txt", "a")
                    coord_file.write("0 {0} {1} {2} {3}\n".format((x + 10) / IMG_SIZE, (y + 10) / IMG_SIZE,
                                                                  w / IMG_SIZE, h / IMG_SIZE))
                    coord_file.close()
                else:
                    coord_file = open(frame_file_name + ".txt", "w")
                    coord_file.write("0 {0} {1} {2} {3}\n".format((x + 10) / IMG_SIZE, (y + 10) / IMG_SIZE,
                                                                  w / IMG_SIZE, h / IMG_SIZE))
                    coord_file.close()

            cv2.rectangle(current_frame, (x, y), (x + w, y + h), (0, 0, 255), 3)


Helpers for data processing

In [ ]:
def cleanup_frames_dir():
    """
    Deletes all files and directories inside YOLO_FRAMES directory
    """
    # Deletes files
    for frame in os.listdir(YOLO_FRAMES):
        file_path = os.path.join(YOLO_FRAMES, frame)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print("Deletion failed {0} due to:\n {1}".format(file_path, e))


def cleanup_data_path_txt():
    """
    Removes all path string from the YOLO_MAIN_DATA YOLO_VALID_DATA YOLO_TEST_DATA
    """
    # Remove the path strings from the main, valid and test files
    open(YOLO_MAIN_DATA, "w").close()
    open(YOLO_VALID_DATA, "w").close()
    open(YOLO_TEST_DATA, "w").close()

In [ ]:
print(os.listdir(YOLO_FRAMES))
print(os.listdir(NUMPY_SAVE))
# When processed data are present, skip
# data processing to prevent unnecessary
# computational time
if len(os.listdir(YOLO_FRAMES)) != 0 and len(os.listdir(NUMPY_SAVE)) != 0:
  SKIP_DATA_PROCESSING = True

if SKIP_DATA_PROCESSING is False:
  cleanup_frames_dir()

## **Allocation of Data into Training and Valid sets**

In [ ]:
# When processed data are present, skip
# data processing to prevent unnecessary
# computational time
if len(os.listdir(YOLO_FRAMES)) != 0 and len(os.listdir(NUMPY_SAVE)) != 0:
  SKIP_DATA_PROCESSING = True

if SKIP_DATA_PROCESSING is False:  
  # Process all the Data files from PATH_DATA
  data_filenames = next(os.walk(os.path.join(PATH_DATA)), (None, None, []))[2]
  data = ()
  list_data = []
  for file in data_filenames:
      print(file)
      data = load_video(PATH_DATA + file)
      list_data.append(data)

In [ ]:
# When processed data are present, skip
# data processing to prevent unnecessary
# computational time
if len(os.listdir(YOLO_FRAMES)) != 0 and len(os.listdir(NUMPY_SAVE)) != 0:
  SKIP_DATA_PROCESSING = True

if SKIP_DATA_PROCESSING is False:
  # Create a numpy array from the processed data
  imaging_data = []
  labels = []
  for data in list_data:
      (x_train, y_train) = data
      imaging_data = imaging_data + x_train
      labels = labels + y_train
  imaging_data = np.array(imaging_data)
  labels = np.array(labels)

  dataset_size = len(labels)

  print("Data Set Size:", dataset_size)
  print("Data: {0},  Labels: {1}".format(imaging_data.shape, labels.shape))

In [ ]:
# When processed data are present, skip
# data processing to prevent unnecessary
# computational time
if len(os.listdir(YOLO_FRAMES)) != 0 and len(os.listdir(NUMPY_SAVE)) != 0:
  SKIP_DATA_PROCESSING = True

if SKIP_DATA_PROCESSING is False:  
  # Randomly allocate data for RES_NET_MODEL and MOBILENET_MODEL
  train_imaging_data, valid_imaging_data, train_labels, valid_labels = train_test_split(imaging_data, labels, test_size=0.2)
  print("train_imaging_data:", train_imaging_data.shape)
  print("train_labels:", train_labels.shape)
  print("valid_imaging_data:", valid_imaging_data.shape)
  print("valid_labels:", valid_labels.shape)
  print("-----------------------------------------------")
  train_imaging_data, test_imaging_data, train_labels, test_labels = train_test_split(train_imaging_data, train_labels, test_size=0.2)
  print("train_imaging_data:", train_imaging_data.shape)
  print("train_labels:", train_labels.shape)
  print("valid_imaging_data:", valid_imaging_data.shape)
  print("valid_labels:", valid_labels.shape)
  print("test_imaging_data:", test_imaging_data.shape)
  print("test_labels:", test_labels.shape)

  np.save(NUMPY_SAVE + "train_imaging_data", train_imaging_data)
  np.save(NUMPY_SAVE + "train_labels", train_labels)
  np.save(NUMPY_SAVE + "valid_imaging_data", valid_imaging_data)
  np.save(NUMPY_SAVE + "valid_labels", valid_labels)
  np.save(NUMPY_SAVE + "test_imaging_data", test_imaging_data)
  np.save(NUMPY_SAVE + "test_labels", test_labels)

In [ ]:
if SKIP_DATA_PROCESSING is False:
  # Clear unnecessary memory
  del train_imaging_data
  del train_labels
  del valid_imaging_data
  del valid_labels
  del test_imaging_data
  del test_labels

In [ ]:
numpy_yolo_frames = np.asarray(os.listdir(YOLO_FRAMES))
print(numpy_yolo_frames.shape)

In [ ]:
yolo_main_set, yolo_validate_set, _, _ = train_test_split(numpy_yolo_frames, numpy_yolo_frames, test_size =0.2)
print("yolo_main_set", yolo_main_set.shape)
print("yolo_validate_set", yolo_validate_set.shape)
print("-----------------------------------------------")
yolo_main_set, yolo_test_set, _, _ = train_test_split(yolo_main_set, yolo_main_set, test_size =0.2)
print("yolo_main_set", yolo_main_set.shape)
print("yolo_validate_set", yolo_validate_set.shape)
print("yolo_test_set", yolo_test_set.shape)

In [ ]:
# Randomly allocate data for validation for YOLO_MODEL
if YOLO_MODEL is True:
    # Updates the data paths for each set of text files
    cleanup_data_path_txt()

    # YOLO_MAIN_DATA
    for main_data in yolo_main_set:
        if main_data.endswith(".jpg"):
            if os.path.isfile(YOLO_MAIN_DATA):
                open(YOLO_MAIN_DATA, "a").write("{0}{1}\n".format(YOLO_FRAMES, main_data))
            else:
                open(YOLO_MAIN_DATA, "w").write("{0}{1}\n".format(YOLO_FRAMES, main_data))

    # YOLO_VALID_DATA
    for valid_data in yolo_validate_set:
        if valid_data.endswith(".jpg"):
            if os.path.isfile(YOLO_VALID_DATA):
                open(YOLO_VALID_DATA, "a").write("{0}{1}\n".format(YOLO_FRAMES, valid_data))
            else:
                open(YOLO_VALID_DATA, "w").write("{0}{1}\n".format(YOLO_FRAMES, valid_data))

    # YOLO_TEST_DATA
    for test_data in yolo_test_set:
        if test_data.endswith(".jpg"):
            if os.path.isfile(YOLO_TEST_DATA):
                open(YOLO_TEST_DATA, "a").write("{0}{1}\n".format(YOLO_FRAMES, test_data))
            else:
                open(YOLO_TEST_DATA, "w").write("{0}{1}\n".format(YOLO_FRAMES, test_data))

# DATA MANIPULATION AND TRAINING FOR ML MODELS
# &
# ANALYTICS FOR ML MODELS

### **YOLO Model**

Setup Darknet framework


In [ ]:
!rm -rf darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
%cd ..

!cd darknet/ && make

In [ ]:
!cd darknet && wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov3.weights

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
!rm darknet/cfg/custom-yolov3-tiny.cfg
!cd darknet && cp cfg/yolov3-tiny.cfg cfg/custom-yolov3-tiny.cfg

%cd darknet/cfg
!sed -i 's/batch=1/batch='{YOLO_BATCH}'/' custom-yolov3-tiny.cfg
!sed -i 's/subdivisions=1/subdivisions='{YOLO_SUBDIVISION}'/' custom-yolov3-tiny.cfg
!sed -i 's/max_batches = 500200/max_batches = '{YOLO_MAX_BATCHES}'/' custom-yolov3-tiny.cfg
!sed -i 's/steps=400000,450000/steps='{YOLO_LOWER_STEPS}','{YOLO_UPPER_STEPS}'/' custom-yolov3-tiny.cfg
!sed -i 's/classes=80/classes='{len(CLASSES)}'/' custom-yolov3-tiny.cfg
!sed -i 's/filters=255/filters='{(len(CLASSES)+5)*3}'/' custom-yolov3-tiny.cfg
%cd /content/

In [ ]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights -P darknet/

In [ ]:
!echo {YOLO_DATA_FILE}
!ls -al darknet/cfg/ | grep -i custom-yolov3-tiny.cfg
!ls -al darknet/ | grep -i yolov3-tiny.weights

Run YOLO model in Darknet framework

In [ ]:
if YOLO_MODEL:
  # Start from fresh
  # !cd darknet && ./darknet detector train {YOLO_DATA_FILE} cfg/custom-yolov3-tiny.cfg yolov3-tiny.weights -dont_show -map

  # Continue from the lastest weights
  !cd darknet && ./darknet detector train {YOLO_DATA_FILE} cfg/custom-yolov3-tiny.cfg {YOLO_BACKUP}custom-yolov3-tiny_last.weights -dont_show -map

### **YOLO Model Loss and mAP Graph**

YOLO model analytics

In [ ]:
try:
  if YOLO_MODEL:
    # img = cv2.imread("darknet/chart.png")
    # cv2_imshow(img)

    image = cv2.imread("darknet/chart.png")
    print(image.shape)
    height, width = image.shape[:2]
    resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()
except AttributeError:
  raise AttributeError("chart.png Has Not Been Detected in darknet/ Directory, Maybe Let the YOLO Model Run More Longer")

In [ ]:
if YOLO_MODEL:
  for yolo_test in yolo_test_set:
    if yolo_test.endswith(".jpg"):
      !cd darknet && ./darknet detector test {YOLO_DATA_FILE} cfg/custom-yolov3-tiny.cfg {YOLO_BACKUP}custom-yolov3-tiny_last.weights {YOLO_FRAMES}{yolo_test}
      
      # Demonstration Purposes Only
      # img = cv2.imread("darknet/predictions.jpg")
      # cv2_imshow(img)

Copy the final model weights for YOLO Model to models/ directory

In [ ]:
!mkdir {FINAL_MODELS_PATH}yolo/
!cp {YOLO_BACKUP}custom-yolov3-tiny_last.weights {FINAL_MODELS_PATH}yolo/

In [ ]:
train_imaging_data = np.load(NUMPY_SAVE + "train_imaging_data.npy")
train_labels = np.load(NUMPY_SAVE + "train_labels.npy")
valid_imaging_data = np.load(NUMPY_SAVE + "valid_imaging_data.npy")
valid_labels = np.load(NUMPY_SAVE + "valid_labels.npy")
test_imaging_data = np.load(NUMPY_SAVE + "test_imaging_data.npy")
test_labels = np.load(NUMPY_SAVE + "test_labels.npy")

In [ ]:
# One-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=len(CLASSES))
valid_labels = tf.keras.utils.to_categorical(valid_labels, num_classes=len(CLASSES))
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(CLASSES))

### **ResNet Model**

In [ ]:
log_dir = "logs/fit/resnet_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
cp_callback = tf.keras.callbacks.ModelCheckpoint(BACKUP_TENSOR + "resnet_model",
                                                  save_weights_only=True,
                                                  verbose=1)

In [ ]:
if RES_NET_MODEL is True:
    model = tf.keras.Sequential(
        [
            tf.keras.applications.ResNet50V2(weights="imagenet",
                                          include_top=False,
                                          input_shape=INPUT_SHAPE,
                                          classes=len(CLASSES)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(len(CLASSES), activation="softmax"),
        ]
    )

    model.summary()

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    
    resnet_train_imaging_data = train_imaging_data
    resnet_train_labels = train_labels

    resnet_valid_imaging_data = valid_imaging_data
    resnet_valid_labels = valid_labels

    # Demonstration Purposes Only
    # cv2_imshow(resnet_train_imaging_data[0])
    # print("Before Shape", resnet_train_imaging_data[0].shape)

    resnet_train_imaging_data = tf.keras.applications.resnet.preprocess_input(resnet_train_imaging_data)
    resnet_valid_imaging_data = tf.keras.applications.resnet.preprocess_input(resnet_valid_imaging_data)

    # Demonstration Purposes Only
    # cv2_imshow(resnet_train_imaging_data[0])
    # print("After Shape", resnet_train_imaging_data[0].shape)

    history_res_net_model = model.fit(resnet_train_imaging_data, resnet_train_labels, 
                                      batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                      validation_data=(resnet_valid_imaging_data, resnet_valid_labels),
                                      callbacks=[tensorboard_callback, cp_callback])

Copy the final model weights for ResNet Model to models/ directory

In [ ]:
if RES_NET_MODEL:
  model.save(FINAL_MODELS_PATH + "resnet_model/resnet_model.h5")
  model = tf.keras.models.load_model(FINAL_MODELS_PATH + "resnet_model/resnet_model.h5")

In [ ]:
if RES_NET_MODEL:
  print(history_res_net_model.history)

 ### ResNet Model Loss Over Epochs

In [ ]:
if RES_NET_MODEL:
  history_res_net_model_dict = history_res_net_model.history
  loss_values = history_res_net_model_dict['loss']
  val_loss_values = history_res_net_model_dict['val_loss']

  epochs = range(1, len(loss_values) + 1)

  plt.plot(epochs, loss_values, 'g', label='Training loss') # blue dots
  plt.plot(epochs, val_loss_values, 'b', label='Validation loss') # blue line
  plt.title("Training and Validation Loss Over Epochs")
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.show()

### ResNet Model Accuracy Over Epochs

In [ ]:
if RES_NET_MODEL:
  acc = history_res_net_model_dict['accuracy']
  val_acc = history_res_net_model_dict['val_accuracy']

  plt.plot(epochs, acc, 'g', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and Validation Accuracy Over Epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()

  plt.show()

### ResNet Model Confusion Matrix

In [ ]:
if RES_NET_MODEL:
  pred_labels = model.predict(test_imaging_data)
  matrix = metrics.confusion_matrix(np.argmax(test_labels, axis=1), np.argmax(pred_labels, axis=1))
  print(matrix)

  plot_confusion_matrix(conf_mat=matrix, figsize=(8, 8), 
                        colorbar=True, show_absolute=False, 
                        show_normed=True,
                        class_names=CLASSES)

In [ ]:
if RES_NET_MODEL:
  # Clear unnecessary memory
  del resnet_train_imaging_data
  del resnet_train_labels
  del resnet_valid_imaging_data
  del resnet_valid_labels

### **MobileNetV2 Model**

In [ ]:
log_dir = "logs/fit/mobilenet_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
cp_callback = tf.keras.callbacks.ModelCheckpoint(BACKUP_TENSOR + "mobilenet_model",
                                                  save_weights_only=True,
                                                  verbose=1)

In [ ]:
if MOBILENET_MODEL is True:
    model = tf.keras.Sequential(
        [
            tf.keras.applications.MobileNetV2(weights="imagenet",
                                          include_top=False,
                                          input_shape=INPUT_SHAPE,
                                          classes=len(CLASSES)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(len(CLASSES), activation="softmax"),
        ]
    )

    model.summary()

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    mobilenet_train_imaging_data = train_imaging_data
    mobilenet_train_labels = train_labels

    mobilenet_valid_imaging_data = valid_imaging_data
    mobilenet_valid_labels = valid_labels

    # Demonstration Purposes Only
    # cv2_imshow(mobilenet_train_imaging_data[0])
    # print("Before Shape: ", mobilenet_train_imaging_data[0].shape)

    mobilenet_train_imaging_data = tf.keras.applications.mobilenet.preprocess_input(mobilenet_train_imaging_data)
    mobilenet_valid_imaging_data = tf.keras.applications.mobilenet.preprocess_input(mobilenet_valid_imaging_data)

    # Demonstration Purposes Only
    # cv2_imshow(mobilenet_train_imaging_data[0])
    # print("After Shape", mobilenet_train_imaging_data[0].shape)

    history_mobilenetv2_model = model.fit(mobilenet_train_imaging_data, mobilenet_train_labels,
                        batches=BATCH_SIZE, epochs=EPOCHS, 
                        validation_data=(mobilenet_valid_imaging_data, mobilenet_valid_labels),
                        callbacks=[tensorboard_callback, cp_callback])

Copy the final model weights for MobileNet Model to models/ directory

In [ ]:
if MOBILENET_MODEL:
  model.save(FINAL_MODELS_PATH + "mobilenet_model/mobilenet_model.h5")
  model = tf.keras.models.load_model(FINAL_MODELS_PATH + "mobilenet_model/mobilenet_model.h5")

In [ ]:
if MOBILENET_MODEL:
  print(history_mobilenetv2_model.history)

 ### MobileNetV2 Model Loss Over Epochs

In [ ]:
if MOBILENET_MODEL:
  history_mobilenetv2_model_dict = history_mobilenetv2_model.history
  loss_values = history_mobilenetv2_model_dict['loss']
  val_loss_values = history_mobilenetv2_model_dict['val_loss']

  epochs = range(1, len(loss_values) + 1)

  plt.plot(epochs, loss_values, 'g', label='Training loss') # blue dots
  plt.plot(epochs, val_loss_values, 'b', label='Validation loss') # blue line
  plt.title("Training and Validation Loss Over Epochs")
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.show()

### MobileNetV2 Model Accuracy Over Epochs

In [ ]:
if MOBILENET_MODEL:
  acc = history_mobilenetv2_model_dict['accuracy']
  val_acc = history_mobilenetv2_model_dict['val_accuracy']

  plt.plot(epochs, acc, 'g', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and Validation Accuracy Over Epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()

  plt.show()

### MobileNetV2 Model Confusion Matrix

In [ ]:
if MOBILENET_MODEL:
  pred_labels = model.predict(test_imaging_data)
  matrix = metrics.confusion_matrix(np.argmax(test_labels, axis=1), np.argmax(pred_labels, axis=1))

  plot_confusion_matrix(conf_mat=matrix, figsize=(8, 8), 
                        colorbar=True, show_absolute=False, 
                        show_normed=True,
                        class_names=CLASSES)

In [ ]:
if MOBILENET_MODEL:
  # Clear unnecessary memory
  del mobilenet_train_imaging_data
  del mobilenet_train_labels
  del mobilenet_valid_imaging_data
  del mobilenet_valid_labels

In [ ]:
# Clear unnecessary memory
del train_imaging_data
del train_labels
del valid_imaging_data
del valid_labels

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# Clean up Logs
# !rm -rf logs/